In [1]:
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google'

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import random
import math
import time
from sklearn.model_selection import GridSearchCV


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


C:\Users\Hudson\anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


The output of torchvision datasets are PILImage images of range [0, 1].
We transform them to Tensors of normalized range [-1, 1].



In [5]:
transform_mnist = transform = transforms.Compose([transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))])

transform_cifar = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [6]:
#create datasets for CIFAR10
train_c10_full= torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_cifar)
test_c10 = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_cifar)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
#create datasets for MNIST
train_mnist_full = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform_mnist)
test_mnist = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform_mnist)

In [8]:
#create datasets for fashion MNIST
train_fmnist_full = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform_mnist)
test_fmnist = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform_mnist)

In [9]:
#this code needs to be in a function , that generates a training set and validation set from an
#input dataset and input minibatch size.  In this case, we call
#train_loader, validation_loader = function(train_c10_full, 32)

# p between 0 and 1, percent of data in tr_loader, q into validationloader
def train_val_split(train_data, minibatch_size, p, q):
    idx_range = list(range(0,len(train_data)))
    random.shuffle(idx_range)
    
    p_idx = int(p * len(idx_range))
    pq_idx = int((p+q) * len(idx_range))
    #middle = len(idx_range)//2
    tr_idx = idx_range[:p_idx]
    val_idx = idx_range[p_idx:pq_idx]
    
    train_subset = torch.utils.data.Subset(train_data, tr_idx)
    val_subset = torch.utils.data.Subset(train_data, val_idx)
    
    tr_loader = torch.utils.data.DataLoader(train_subset, batch_size=minibatch_size,
                                          shuffle=True, num_workers=2)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=minibatch_size,
                                          shuffle=True, num_workers=2)
    return tr_loader, val_loader


In [10]:
#CIFAR10 dataloaders
train_c10_loader, val_c10_loader = train_val_split(train_c10_full, 32, 0.8, 0.2)


train_c10_mini, val_c10_mini = train_val_split(train_c10_full, 32, 0.05, 0.05)


test_c10_loader = torch.utils.data.DataLoader(test_c10, batch_size=32,
                                         shuffle=False, num_workers=2)


#MNIST dataloaders
train_mnist_loader,  val_mnist_loader = train_val_split(train_mnist_full, 32, 0.8, 0.2)

test_mnist_loader = torch.utils.data.DataLoader(test_mnist, batch_size=32,
                                         shuffle=False, num_workers=2)

#Fashion-MNIST dataloaders
train_fmnist_loader,  val_fmnist_loader = train_val_split(train_fmnist_full, 32, 0.8, 0.2)

test_fmnist_loader = torch.utils.data.DataLoader(test_fmnist, batch_size=32,
                                         shuffle=False, num_workers=2)

c10_classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
mnist_classes = ('0','1','2','3','4','5','6','7','8','9')

fmnist_classes = ('t-shirt/top', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot')

In [11]:
print(train_fmnist_full)


Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=0.5, std=0.5)
           )


In [12]:
#un-normalize and display an imaage from a dataset
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()


In [13]:
#Adapted from google's tutorial
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    :param cm: (numpy matrix) confusion matrix
    :param classes: [str]
    :param normalize: (bool)
    :param title: (str)
    :param cmap: (matplotlib color map)
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 8))   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [14]:
#show_sample will display the first minibatch of dataloader dl
#with the first 4 images labeled according to dl_classes
def show_samples(dl, dl_classes):

    # get some random training images
    dataiter = iter(dl)
    images, labels = dataiter.next()

    # show images
    imshow(torchvision.utils.make_grid(images))
    # print labels
    print(' '.join('%5s' % dl_classes[labels[j]] for j in range(4)))


In [15]:
#show_samples(test_fmnist_loader, fmnist_classes)
#show_samples(test_c10_loader, c10_classes)

In [16]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# This section is where we explore different NN architectures

In [17]:
class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
print(MNISTNet())

In [18]:
class c10Net(nn.Module):
    def __init__(self):
        super(c10Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [19]:
print('hey bud')

hey bud


In [20]:
criterion = nn.CrossEntropyLoss()

In [21]:
#compute the accuracy of net net identifying the classes of data in the dataloader dl
def compute_accuracy(net, dl):
    correct = 0
    total = 0
    net.to(device)
    with torch.no_grad():
        for (images, labels) in dl:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return 100 * correct / total

In [22]:
#train net with a specified number of
def train_net_bounded(net, hyper, trainloader, validationloader, print_mode = False):
    (num_epochs, learning_rate, momentum) = hyper
    
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
    train_loss_list = []
    val_loss_list = []
    train_start = time.time()

    net = net.to(device)
    
    # loop until the validation accuracy stagnates for 5 epochs
    for epoch in range(num_epochs):
        tot_train_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            tot_train_loss += loss.item()


        
        # train_acc = compute_accuracy(net, trainloader)
        # val_acc = compute_accuracy(net, validationloader)
        
        tot_val_loss = 0.0
        with torch.no_grad():
            for (images, labels) in validationloader:
                images, labels = images.to(device), labels.to(device)
                predictions = net(images)
                batch_val_loss = criterion(predictions, labels)
                tot_val_loss += batch_val_loss.item() 



        avg_train_loss = tot_train_loss / len(trainloader)
        avg_val_loss = tot_val_loss / len(validationloader)
        current = time.time()
        
        if(print_mode):
            print('Epoch: %d Train Loss: %f Val Loss: %f Time: %ds' % \
                 (epoch,avg_train_loss,avg_val_loss, current-train_start))

        train_loss_list.append(avg_train_loss)
        val_loss_list.append(avg_val_loss)
        
    train_end = time.time()
    
    print('Finished Training')
    best_epoch = val_loss_list.index(min(val_loss_list)) + 1
    print('Best model at epoch: %d with validation loss of %f'  % (best_epoch, val_loss_list[best_epoch - 1]))
    return(train_loss_list, val_loss_list, train_end-train_start)

In [23]:
#a modification of the above, where we train until the validation accuracy stops improving.
#Will it work?
def train_net_unbounded(net, hyper, trainloader, validationloader,path, print_mode = False):
    (learning_rate, momentum) = hyper
    
    optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
    train_loss_list = []
    val_loss_list = []
    train_start = time.time()

    net = net.to(device)
    
    best_val_loss = 100 #arbitrarily large number
    epoch = 0
    # loop until the validation accuracy stagnates for 5 epochs
    while (len(val_loss_list) < 20 or val_loss_list[epoch-1] < val_loss_list[epoch - 10]):
        tot_train_loss = 0.0
        for i, (inputs, labels) in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            tot_train_loss += loss.item()

        epoch+=1
        
        # train_acc = compute_accuracy(net, trainloader)
        # val_acc = compute_accuracy(net, validationloader)
        
        tot_val_loss = 0.0
        with torch.no_grad():
            for (images, labels) in validationloader:
                images, labels = images.to(device), labels.to(device)
                predictions = net(images)
                batch_val_loss = criterion(predictions, labels)
                tot_val_loss += batch_val_loss.item() 



        avg_train_loss = tot_train_loss / len(trainloader)
        avg_val_loss = tot_val_loss / len(validationloader)
        current = time.time()
        
        if (avg_val_loss < best_val_loss):
            best_val_loss = avg_val_loss
            torch.save(net.state_dict(), path)

        if(print_mode):
            print('Epoch: %d Train Loss: %f Val Loss: %f Time: %ds' % \
                 (epoch,avg_train_loss,avg_val_loss, current-train_start))

        train_loss_list.append(avg_train_loss)
        val_loss_list.append(avg_val_loss)
        
    train_end = time.time()
    
    print('Finished Training after %fs' % (train_end - train_start))
    best_epoch = val_loss_list.index(min(val_loss_list)) + 1
    print('Best model at epoch: %d with validation loss of %f'  % (best_epoch, val_loss_list[best_epoch - 1]))
    return(train_loss_list, val_loss_list, train_end-train_start)

In [24]:
net = c10Net()
save_name = 'lanet_cifar1.pth'
hyper = (0.01,0.9) #(learning_rate, momentum)
train_loss_list, validation_loss_list, t = train_net_unbounded(net, hyper, train_c10_mini, val_c10_mini, save_name, True)
#t1, v1 = train_acc_list, validation_acc_list

Epoch: 1 Train Loss: 2.301781 Val Loss: 2.298384 Time: 11s
Epoch: 2 Train Loss: 2.292041 Val Loss: 2.278740 Time: 25s


KeyboardInterrupt: 

#Now we grid search over our hyperparameters:


In [27]:
params = {
    'lr': [0.1, 0.05, 0.01, 0.001],
    'm' : [0.7, 0.8, 0.9, 0.95]
}
params_dummy = {
    'lr': [0.1],
    'm' : [0.7]
}
print(type(params))

<class 'dict'>


In [28]:
def grid_search_net(net, trainloader, validationloader, out_path, params):
    start = time.time()
    temp_path = "temp.pth"
    lrs, ms = params['lr'], params['m']
    best_val_loss, best_t, best_trList, best_valList, best_lr, best_m = 100, 0, [], [], 0, 0
    for lr in lrs:
        for m in ms:
            print('Trying Learning Rate %f and Momentum %f' % (lr,m))
            trList, valList, t = train_net_unbounded(net, (lr,m), trainloader, validationloader, temp_path, False)
            if min(valList) < best_val_loss:
                best_val_loss, best_t, best_trList, best_valList, best_lr, best_m = min(valList), t, trList, valList, lr, m
                net.load_state_dict(torch.load(temp_path))
                torch.save(net.state_dict(), out_path)
    end = time.time()
    print('Best Validation Loss: %f' % best_val_loss)
    print('Best Learning Rate: %f' % best_lr)
    print('Best Momentum: %f' % best_m)
    print('Total time elapsed %ds' % (end - start))
    return best_trList, best_valList, best_t, best_lr, best_m






In [ ]:
x = c10Net()
out = 'grid_search_lanet_cifar10.pth'
tr, val, t, lr, m = grid_search_net(x, train_c10_mini, val_c10_mini, out, params_dummy)


Trying Learning Rate 0.100000 and Momentum 0.700000


In [ ]:
# PATH = './cifar_net.pth'
# PATH_MNIST = './mnist_net.pth'
# torch.save(net.state_dict(), PATH_MNIST)

In [ ]:
# net = c10Net()
# net.load_state_dict(torch.load(out))

In [ ]:
def plot_losses(train_history, val_history, t, hyper):

    colors = ['r','b']
    
    x = np.arange(1, len(train_history) + 1)

    plt.figure(figsize=(8, 6))
    plt.plot(x, train_history, color=colors[0], label="Training loss", linewidth=2)
    plt.plot(x, val_history, color=colors[1], label="Validation loss", linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.title("Evolution of the training and validation loss")
    plt.show()
    (lr, m) = hyper
    print('Elapsed: %ds Learning Rate: %d Momentum: %d'% (t, lr, m))

plot_losses(train_loss_list, validation_loss_list, t, hyper)


# This next section is adapted directly from a Google Colab tutorial
The tutorial can be found here: https://colab.research.google.com/drive/1B5KQvPySqYEa6XicRHdOwgv8fN1BrCgQ


In [ ]:
#load in a dataset that was saved earlier


In [ ]:
#ADAPTED FROM GOOGLE COLAB PYTORCH TUTORIAL
def dataset_accuracy(net, data_loader, name=""):
    net = net.to(device)
    correct = 0
    total = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    accuracy = 100 * float(correct) / total
    print('Accuracy of the network on the {} {} images: {:.2f} %'.format(total, name, accuracy))

def compute_accuracy(net, train, val, test):
    dataset_accuracy(net, train, "training")
    dataset_accuracy(net, val, "validation")
    dataset_accuracy(net, test, "testing")

In [ ]:
compute_accuracy(net, train_c10_loader, val_c10_loader, test_c10_loader)

In [ ]:
#FROM GOOGLE COLAB TUTORIAL
def accuracy_per_class(net, classes, testloader):
    net = net.to(device)
    # (real, predicted)
    confusion_matrix = np.zeros((len(classes), len(classes)), dtype=np.int64)

    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(16):
            confusion_matrix[labels[i], predicted[i]] += 1
            label = labels[i]

    print("{:<10} {:^10}".format("Class", "Accuracy (%)"))
    for i in range(len(classes)):
        class_total = confusion_matrix[i, :].sum()
        class_correct = confusion_matrix[i, i]
        percentage_correct = 100.0 * float(class_correct) / class_total
        
        print('{:<10} {:^10.2f}'.format(classes[i], percentage_correct))
    return confusion_matrix

confusion_matrix = accuracy_per_class(net, c10_classes, test_c10_loader)

In [ ]:
# Plot normalized confusion matrix
plot_confusion_matrix(confusion_matrix, c10_classes, normalize=True,
                      title='Normalized confusion matrix')

# Plot non-normalized confusion matrix
plot_confusion_matrix(confusion_matrix, c10_classes,
                      title='Confusion matrix, without normalization')

Okay, so what next?

How do we run these neural networks on the GPU?

Training on GPU
----------------
Just like how you transfer a Tensor onto the GPU, you transfer the neural
net onto the GPU.

Let's first define our device as the first visible cuda device if we have
CUDA available:



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

The rest of this section assumes that ``device`` is a CUDA device.

Then these methods will recursively go over all modules and convert their
parameters and buffers to CUDA tensors:

.. code:: python

    net.to(device)


Remember that you will have to send the inputs and targets at every step
to the GPU too:

.. code:: python

        inputs, labels = data[0].to(device), data[1].to(device)

Why dont I notice MASSIVE speedup compared to CPU? Because your network
is really small.

**Exercise:** Try increasing the width of your network (argument 2 of
the first ``nn.Conv2d``, and argument 1 of the second ``nn.Conv2d`` –
they need to be the same number), see what kind of speedup you get.

**Goals achieved**:

- Understanding PyTorch's Tensor library and neural networks at a high level.
- Train a small neural network to classify images

Training on multiple GPUs
-------------------------
If you want to see even more MASSIVE speedup using all of your GPUs,
please check out :doc:`data_parallel_tutorial`.

Where do I go next?
-------------------

-  :doc:`Train neural nets to play video games </intermediate/reinforcement_q_learning>`
-  `Train a state-of-the-art ResNet network on imagenet`_
-  `Train a face generator using Generative Adversarial Networks`_
-  `Train a word-level language model using Recurrent LSTM networks`_
-  `More examples`_
-  `More tutorials`_
-  `Discuss PyTorch on the Forums`_
-  `Chat with other users on Slack`_


